In [69]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [70]:
adata_norm_exp = sc.read_h5ad("norm_exp.h5ad")
print(adata_norm_exp)

AnnData object with n_obs × n_vars = 5577480 × 300
    obs: 'gw', 'sample', 'region', 'H1_cluster', 'H2_cluster', 'H3_cluster', 'H1_annotation', 'H2_annotation', 'H3_annotation', 'area', 'layer'
    obsm: 'spatial'


/Users/cyluo/anaconda3/envs/xuyuEnv/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [71]:
area_of_interest = ['V1', 'V2', 'A-V1', 'A-V2', 'B-V1', 'B-V2']
adata_norm_exp_subset = adata_norm_exp[adata_norm_exp.obs['area'].isin(area_of_interest)].copy()
adata_V1V2 = adata_norm_exp_subset[adata_norm_exp_subset.obs['gw'].isin(['gw15'])].copy()
adata_V1V2 = adata_V1V2[adata_V1V2.obs['sample'].isin(['UMB1367'])].copy()
adata_V1V2 = adata_V1V2[adata_V1V2.obs['region'].isin(['O1'])].copy()
adata_V1V2 = adata_V1V2[adata_V1V2.obs['H1_annotation'].isin(['EN-Mig'])].copy()

def transform_value(value):
    if value == 'V1' or value == 'A-V1' or value == 'B-V1':
        return 'V1'
    elif value == 'V2' or value == 'A-V2' or value == 'B-V2':
        return 'V2'

# Create new column A based on column B
adata_V1V2.obs['area_new'] = adata_V1V2.obs['area'].apply(transform_value)

print(adata_V1V2.obs.head())

                       gw   sample region  H1_cluster  H2_cluster  H3_cluster  \
cell                                                                            
2277265800298100216  gw15  UMB1367     O1           3          16          83   
2277265800298100237  gw15  UMB1367     O1           3          16          82   
2277265800298100240  gw15  UMB1367     O1           3          16          83   
2277265800298100246  gw15  UMB1367     O1           3          16          83   
2277265800298100291  gw15  UMB1367     O1           3          16          80   

                    H1_annotation H2_annotation H3_annotation  area layer  \
cell                                                                        
2277265800298100216        EN-Mig         EN-L2       EN-L2-3  B-V2    l4   
2277265800298100237        EN-Mig         EN-L2       EN-IZ-3  B-V2    l4   
2277265800298100240        EN-Mig         EN-L2       EN-L2-3  B-V2    l4   
2277265800298100246        EN-Mig         EN-L2

In [72]:
unique_count = adata_V1V2.obs['area_new'].unique()
print(unique_count)

['V2', 'V1']
Categories (2, object): ['V1', 'V2']


In [73]:
# Perform DE analysis
sc.tl.rank_genes_groups(adata_V1V2, groupby='area_new') 

In [74]:
result = adata_V1V2.uns['rank_genes_groups']
de_df = pd.DataFrame({
    'gene': result['names']['V1'], 
    'log_fold_change': result['logfoldchanges']['V1'],
    'pvals': result['pvals']['V1'],
    'pvals_adj': result['pvals_adj']['V1']
})
de_df['-log10_pval'] = -np.log10(de_df['pvals_adj'])

/Users/cyluo/anaconda3/envs/xuyuEnv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [75]:
de_df.to_csv('DEG_EN_Mig.csv')